In [1]:
# Basic
from datetime import datetime, timedelta
import time

# Data Analysis Specific
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse as date_parser
from geopy.geocoders import Nominatim

# Marchine Learning Specific
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

# IPython magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
DATA_PATH = './data/'
DETROIT_LAT = (42.252, 42.452)
DETROIT_LNG = (-83.295, -82.895)
SIZE_RATIO = 1000
DETROIT_WIDTH = 200
DETROIT_HEIGHT = 400
print DETROIT_WIDTH, DETROIT_HEIGHT, float(DETROIT_WIDTH)/DETROIT_HEIGHT

200 400 0.5


## The naive (grid) building method can not get good result, partially due to the naive method didn't consider the space relationship. 

## The grid method can be used if using convolutional neural network (just an assumption). For time limit, I didn't explore this idea. I might make this trial later.

## Now, it may be a good idea to use clustering (in fact this is the first idea I got), on the space only, to capture the space relationships.

## There are many clustering algos, I will first try kmeans, for simpleness.

In [3]:
clean_permit = pd.read_csv('clean/permit.csv',index_col=0)
clean_violation = pd.read_csv('clean/violation.csv',index_col=0)
clean_crime = pd.read_csv('clean/crime.csv',index_col=0)
clean_311 = pd.read_csv('clean/311.csv',index_col=0)

In [4]:
clean_crime.date.isnull().sum()

0

In [5]:
clean_permit.date.isnull().sum()

3

In [6]:
clean_permit[~clean_permit['turst']]

,lat,lng,date,addr,turst
671,NaN,NaN,2015/7/30,8045 BRACE,False
672,NaN,NaN,2015/7/30,8909 FAUST,False
676,NaN,NaN,2015/7/30,7290 CLAYBURN,False
678,NaN,NaN,2015/7/30,6894 EVERGREEN,False
680,NaN,NaN,2015/7/30,8649 ARTESIAN,False
681,NaN,NaN,2015/7/30,8221 ASHTON,False
683,NaN,NaN,2015/7/30,9601 AUBURN,False
684,NaN,NaN,2015/7/30,12099 AUBURN,False
685,NaN,NaN,2015/7/30,7340 BRACE,False
687,NaN,NaN,2015/7/30,7458 CLAYBURN,False


In [7]:
clean_violation.date.isnull().sum()

38854

In [8]:
clean_311.date.isnull().sum()

0

In [9]:
len(clean_violation.date.dropna().sort_values())

268950

In [10]:
def dropna(df):
    df = df.dropna()
    if hasattr(df,"turst"):
        df = df[df.turst]
    return df

In [11]:
addr = dropna(clean_violation)[['lat','lng']]

In [12]:
from sklearn.preprocessing import MinMaxScaler
class Scaler(MinMaxScaler):
    def transform(self,X):
        return pd.DataFrame(super(Scaler,self).transform(X),index=X.index,columns=X.columns)
scaler = Scaler().fit(addr)

In [13]:
addr = scaler.transform(addr)

# Give up using sklearn... too slow. Use Graphlab Create instead.

In [ ]:
from sklearn.cluster import KMeans
cluster = KMeans(n_clusters=20000,max_iter=300)
cluster.fit(addr)

In [ ]:
plt.scatter(*zip(*cluster.cluster_centers_))

In [ ]:
export = pd.DataFrame(cluster.cluster_centers_,oolumns=addr.columns)

In [ ]:
export.to_csv("center.csv")